# Visualizing Illinois:
Including none cleaned:
* approved state senate plan districts
* approved congressional districts

@authors: vcle, bpuhani

## Loading needed libraries

In [29]:
import sys
import time
import warnings
import io
import os

import maup
import pandas as pd
from matplotlib import pyplot as plt

import utilities as util

import networkx as nx
from gerrychain import Graph, GeographicPartition


In [30]:
maup.progress.enabled = False

warnings.filterwarnings('ignore')

start_time = time.time()

### configuration to capture print output
original_stdout = sys.stdout

## Downloading the data

All data retrieved in April 2025: <br>
| **Dataset**                                                                                                        | **Description**                                                                           | **Metadata**                                                                                      |
|--------------------------------------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------|
| [2021 State Senate District plan](https://redistrictingdatahub.org/dataset/2021-oregon-state-senate-adopted-plan/) | 2021 Illinois State Senate Approved Plan                                                  | [Link](https://redistrictingdatahub.org/wp-content/uploads/2021/12/readme_il_sldu_2021.txt)       |
| [2021 Congressional District plan](https://redistrictingdatahub.org/dataset/2021-illinois-congressional-districts-approved-plan/) | 2021 Illinois Congressional Districts                                                     | [Link](https://redistrictingdatahub.org/wp-content/uploads/2021/11/readme_il_cong_adopted_2021.txt)        |

## Loading the needed data.
This notebook requires running:
* `0_IL_import_and_explore_data.ipynb`

before this one.

In [31]:
# rename files to mitigate name issues

old_base = "il_data/il_cong_adopted_2021/HB 1291 FA #1"
new_base = "il_data/il_cong_adopted_2021/HB_1291_FA_1"

if os.path.exists(old_base):
    for ext in [".shp", ".dbf", ".shx", ".prj"]:
        old = old_base + ext
        new = new_base + ext
        if os.path.exists(old):
            os.rename(old, new)

In [32]:
# Paths to the data
sen_path = "il_data/il_sldu_2021/il_sldu_2021.shp"
cong_path = "il_data/il_cong_adopted_2021/HB_1291_FA_1.shp"

In [33]:
sen_df = util.load_shapefile(sen_path)
cong_df = util.load_shapefile(cong_path) # throws error

Loading shapefile from il_data/il_sldu_2021/il_sldu_2021.shp...
Shapefile data loaded from cache.
Loading shapefile from il_data/il_cong_adopted_2021/HB_1291_FA_1.shp...
Shapefile data loaded from cache.


Run `maup.doctor` for both senate plan and congressional plan

In [34]:
try:
    print(maup.doctor(sen_df))
except Exception as e:
    print(f"Error in MAUP Doctor for sen_df: {e}")

True


In [35]:
try:
    print(maup.doctor(cong_df))
except Exception as e:
    print(f"Error in MAUP Doctor for cong_df: {e}")

True


So far so good, all clean

### Run `maup.doctor` for each of the district in the approved senate plan

In [36]:
# we know that Illinois has 59 districts
nr_of_districts = len(set(sen_df["DISTRICTN"]))
assert (nr_of_districts == 59), f"Number of state senate districts in Illinois: {nr_of_districts} != 59"

state_senate_districts_with_issues = {}

# Create a StringIO object to capture the output
captured_output = io.StringIO()

for district in set(sen_df["DISTRICTN"]):
    district_df = sen_df[sen_df["DISTRICTN"] == district]
    print("Checking district: ", district)
    success: bool = False
    output: str = ""

    try:
        # Redirect stdout to the StringIO object
        sys.stdout = captured_output

        try:
            # maup.doctor(district_df) will return True if there are no issues
            success = maup.doctor(district_df)
        except Exception as e:
            print(f"Error in MAUP Doctor for district {district}: {e}")

        # Get the captured output as a string without new lines
        output = captured_output.getvalue().replace("\n", "")

    finally:
        # Restore the original stdout
        sys.stdout = original_stdout
        captured_output.truncate(0) # Clear the StringIO buffer
        captured_output.seek(0) # Reset the cursor to the beginning
        if not success:
            print(f"Problem in district {district}: {output}")
            state_senate_districts_with_issues[district] = output

captured_output.close()

Checking district:  1
Checking district:  2
Checking district:  3
Checking district:  4
Checking district:  5
Checking district:  6
Checking district:  7
Checking district:  8
Checking district:  9
Checking district:  10
Checking district:  11
Checking district:  12
Checking district:  13
Checking district:  14
Checking district:  15
Checking district:  16
Checking district:  17
Checking district:  18
Checking district:  19
Checking district:  20
Checking district:  21
Checking district:  22
Checking district:  23
Checking district:  24
Checking district:  25
Checking district:  26
Checking district:  27
Checking district:  28
Checking district:  29
Checking district:  30
Checking district:  31
Checking district:  32
Checking district:  33
Checking district:  34
Checking district:  35
Checking district:  36
Checking district:  37
Checking district:  38
Checking district:  39
Checking district:  40
Checking district:  41
Checking district:  42
Checking district:  43
Checking district:  

In [37]:
print("There are a total of", len(state_senate_districts_with_issues), "districts with issues.")

There are a total of 0 districts with issues.


### Run `maup.doctor` for each of the districts in the congressional plan

In [38]:
# we know that Illinois has 17 congressional districts
nr_of_districts = len(set(cong_df["DISTRICTN"]))
assert (nr_of_districts == 17), f"Number of congressional districts in Illinois: {nr_of_districts} != 59"

cong_districts_with_issues = {}

# Create a StringIO object to capture the output
captured_output = io.StringIO()

for district in set(cong_df["DISTRICTN"]):
    district_df = cong_df[cong_df["DISTRICTN"] == district]
    print("Checking district: ", district)
    success: bool = False
    output: str = ""

    try:
        # Redirect stdout to the StringIO object
        sys.stdout = captured_output

        try:
            # maup.doctor(district_df) will return True if there are no issues
            success = maup.doctor(district_df)
        except Exception as e:
            print(f"Error in MAUP Doctor for district {district}: {e}")

        # Get the captured output as a string without new lines
        output = captured_output.getvalue().replace("\n", "")

    finally:
        # Restore the original stdout
        sys.stdout = original_stdout
        captured_output.truncate(0) # Clear the StringIO buffer
        captured_output.seek(0) # Reset the cursor to the beginning
        if not success:
            print(f"Problem in district {district}: {output}")
            cong_districts_with_issues[district] = output

captured_output.close()

Checking district:  1
Checking district:  2
Checking district:  3
Checking district:  4
Checking district:  5
Checking district:  6
Checking district:  7
Checking district:  8
Checking district:  9
Checking district:  10
Checking district:  11
Checking district:  12
Checking district:  13
Checking district:  14
Checking district:  15
Checking district:  16
Checking district:  17


In [39]:
print("There are a total of", len(cong_districts_with_issues), "congressional districts with issues.")

There are a total of 0 congressional districts with issues.
